# Filter Hartford crashes that happened between November through May of all years

In [1]:
import pandas as pd
from geopy.distance import distance, lonlat
from tqdm import tqdm

In [2]:
# Read Persons file
#persons_path = 'raw/2019-06-17/export_2859_2.csv'
#persons_raw = pd.read_csv(persons_path, engine="python", skiprows=1, dtype=str)

# Read Crashes file
crashes_path = 'raw/2019-06-17/export_2859_0.csv'
crashes_raw = pd.read_csv(crashes_path, engine="python", skiprows=1, dtype=str)

In [3]:
# We only care about Hartford crashes that happened between November and May (inclusive)
relevant_crashes = crashes_raw[ (crashes_raw['Town Name'] == 'Hartford')
                               & crashes_raw['Month'].isin(['11', '12', '1', '2', '3', '4', '5'])]

In [4]:
# Add nearest speed hump to the relevant Crashes dataset
speedhumps = pd.read_csv('data/speed-humps-all-locations.csv')

def get_nearest_speedhump(crash):
    crash_coords = (crash['Longitude'], crash['Latitude'])
    hump_type = 99999
    hump_dist = 99999
    hump_date = 99999
    
    for i, row in speedhumps.iterrows():
        speedhump_coords = (row['Long'], row['Lat'])
        dist = distance(lonlat(*crash_coords), lonlat(*speedhump_coords)).feet
        if dist < hump_dist:
            hump_dist = dist
            hump_type = row['SpeedhumpType']
            hump_date = row['InstallDate']
            
    return pd.Series({'NearestSpeedhump': hump_type, 'NearestSpeedhumpDistance': hump_dist, 'NearestSpeedhumpInstallDate': hump_date})

tqdm.pandas()
crashes_with_humps = pd.concat([relevant_crashes, relevant_crashes.progress_apply(get_nearest_speedhump, axis=1)], axis=1)

100%|██████████| 18521/18521 [24:35<00:00, 12.63it/s]


In [5]:
# Create Period column based on the date (eg '2015/2016' for
# crashes that  happened between November 2015 and May 2016)
def get_period(date):
    if (date >= '2015-11-01') & (date <= '2016-05-31'):
        return '2015/2016'
    if (date >= '2016-11-01') & (date <= '2017-05-31'):
        return '2016/2017'
    if (date >= '2017-11-01') & (date <= '2018-05-31'):
        return '2017/2018'
    if (date >= '2018-11-01') & (date <= '2019-05-31'):
        return '2018/2019'
    return 'None'

crashes_with_humps['Period'] = crashes_with_humps['Date Of Crash'].apply(get_period)

### Save relevant data to the output .csv file

In [6]:
crashes_with_humps[crashes_with_humps['Period'] != 'None'].filter([
    'CrashId',
    'Date Of Crash',
    'Period',
    'Crash Severity Text Format',
    'Most Severe Injury Text Format',
    'Number Of Motor Vehicles',
    'Road Description',
    'Manner of Crash / Collision Impact Text Format',
    'Weather Condition Text Format',
    'Latitude', 'Longitude',
    'NearestSpeedhump', 'NearestSpeedhumpDistance', 'NearestSpeedhumpInstallDate'
]).to_csv('output/crashes-with-humps.csv', index=False)